<a href="https://colab.research.google.com/github/Benendead/LSTMjazz/blob/master/Research/Summaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Topics Left To Study** (Paper Mentioned In)  
* RTRL error (Eck 2002)
* BPTT (Eck 2002, Lackner 2016)
* Kalman filter to update weights? (Eck 2002)
* [RBM](http://imonad.com/rbm/restricted-boltzmann-machine/) (Bickerman 2010)
* [Contrastive Divergence](http://www.cs.toronto.edu/~fritz/absps/tr00-004.pdf) (Bickerman 2010)
* Fourier Transform (Lackner 2016, Hilscher 2018)
* [Simulated annealing](https://cs.stackexchange.com/questions/79241/what-is-temperature-in-lstm-and-neural-networks-generally) (Hilscher 2018)
* Autocorrelation w/ FFT (Hilscher 2018)
* Seq2Seq (Agarwala 2017)
* Bidirectionality, attention (Agarwala 2017)
* Temperature parameters (Hilscher 2018, Brunner 2017)

# Summaries

## 1. Choi et al. (2016)

**Summary**  
They used char-RNN and word-RNN to generate jazz chord progressions. Their data was 2,486 scores transposed to C from real books. The models were fed text data of chord progressions, with all quarter notes filled in, as in:  
-START- F:9 F:9 F:9 F:9 D:min7 D:min7 G:9 G:9  
C:maj C:maj F:9 F:9 C:maj C:maj C:maj C:maj -END-  
There were 39 distinct characters, 1,259 distinct chords, and 539,609 chords overall.

**Why char-RNN?**  
Benefits of char-RNN:
* Smaller vocabulary
* Fewer assumptions about music made

Downsides of char-RNN:
* Shorter effective length of memory, as more time steps are needed to input the same information

**Model Architecture**  
Two LSTM layers with 512 hidden units (hidden state dimensionality) and then Dropout of 0.2 after each LSTM layer. Put together in Keras with categorical cross entropy as loss, Adam optimizer, and stochastic prediction based on a diversity parameter $\alpha$. New probabilities are calculated:  
$\hat{p}_i=e^{log(p_i)/\alpha}$, where $p_i$ is the probability for the $i$ states.  
A state is then selected based on the probabilities.

**Successes**  
Their char-RNN learned $\text{ii}-\text{V}-\text{I}$, passing chords, modal interchange chords, and tritone substitutions. The word-RNN included modal interchange, circle of fifths progressions, and descending basslines. Their code is available [here](https://github.com/keunwoochoi/lstm_real_book).  
*-Fin 1/24/2019-*

## 2. Eck et al. (2002)

**Summary**  
This paper ran two experiments with LSTMs:
1. Memorizing a chord progression.
2. Also improvising pentatonic blues over a memorized progression.  

Both experiments were successful using relatively simple LSTMs. They represented the data as a vector with 12 possible chord notes and 13 melody notes. Their quantization step was an eighth note and note representation was a simple 1/0 on/off input.

Examples can be found [here](https://web.archive.org/web/20190104192500/http://people.idsia.ch/~juergen/blues/).

**Experiment 1**  
The data was just the same progression repeatedly. The network architecture used 4 LSTM cells each with 2 hidden cells all fully connected to each other and the input layer. Forget, input, and output gate biases were set to -0.5, -1.0, -1.5, and -2.0 for the four blocks, respectively. Output biases were 0.5, learning rate 0.00001, momentum rate 0.9. Loss was cross-entropy and output function was logistic sigmoid.

In the output, chord notes were on if they had a probability above 0.5. Training ended once the chord sequence was completely memorized.

**Experiment 2**  
Some cell blocks learned melody and others learned chords. There were eight cell blocks total with two hidden cells each. Melody information never reached chord cell blocks, although everything else was identical to Experiment 1.

Again, chord outputs had a 0.5 threshold. The melody output needed to pick one note, so the probabilities were constrained to sum to 1 and then a random number in [0,1] was used. Training ended until objective error plateaued. Outputs were seeded and then allowed to compose progressions.  
*-Fin 10/24/2019-*

## 3. Bickerman et al. (2010)

**Summary**  
This work used Deep Belief Networks (DBNs) to compose monophonic jazz music given some chord sequence. Their data was a corpus of 4-bar jazz licks cycling over the $\text{ii}-\text{V}-\text{I}-\text{VI}^7$ turnaround progression. They transposed these progressions up 0, 1, 2, and 3 semitones and the model was still able to produce chord-compatible melodies. Code is [here](https://sourceforge.net/projects/rbm-provisor/).

**Data Representation**  
They took the 4-bar examples and encoded them as bit vectors with 12 time steps (or "slots") per beat. Each time step had 30 bits split up as:
* 18 melody bits with 12 indicating the note's pitch (1-hot), 4 indicating octave (1-hot), 1 for sustained note, and 1 for rests.
* 12 chord bits with 1/0 indicating on/off for each possible note in a chord.

These two parts were concatenated to form one time step's input.

**DBNs**  
The DBN is simply a multi-layer RBM. An input layer is set to the input, bi-directional weights are used to calculate the fully-connected hidden layer, and then the same weights are used to set the input layer. This repeats until stabilization. DBNs just extend this by adding more hidden layers, which allows them to learn features of learned features.

Music was generated by "clamping" the chord bits down from an input, giving a random melody seed, and letting the trained network stabilize for the next few beats of melody. The input is shifted down one beat, the past beat of melody is clamped, and the process repeats. To select which notes to play, the melody uses only the highest probability of pitch and octave and chooses to play that, sustain, or rest.

The authors found that a DBN with 4 layers total of 1441 input nodes (4 beats, 12 slots each with 30 bits/slot plus 1 bias bit) and 750, 375, and 200 hidden nodes. Training lasted 250 epochs over the 100 4-bar licks.

**Results**  
The generated melodies were mostly chord tones with some color, while foreign notes were avoided. Large jumps, octaves, half-step approach tones, and triplets were all avoided by the DBN. Repeated notes were overly common in the generated music.  
*-Fin 1/25/2019-*

## 4. Lackner (2016)

**Summary**  
This paper tested multiple LSTM architectures with the goal of composing monophonic melodies for given 8-bar chord sequences.

**Dataset and Representation**  
The LSTMs were trained on 68 8-bar sections of Beatles music from the book "Pop Classics For Piano: The Very Best Of The Beatles - Easy Arrangements for Piano." The chord notes ranged from C2-B2 and the melodies' from C3-B4. Quantization was to the 16th note.

For the data representation, MIDI was used and segmented into a piano roll matrix. Notes were 1/0 on/off with the second half of any note indicated by a vector of all 0s. Thus the matrix was size $(2*(\text{# timesteps}), \text{note range})$. This matrix was sampled into $n$-timestep-long sequences of chord data. Each sample was given a label with the melody information for the $n+1$ timestep. The number of samples, $S$, was the total length of the data (in timesteps) minus $n+1$, as the penultimate time step needs the ultimate for a target vector.

Thus the data was input into Keras as an input matrix of size $(S, n, |\text{chord pitch range}|)$. The target matrix was then $(S, |\text{melody pitch range}|)$. The number of samples was the total length (in timesteps) of the data minus $n$, the sequence length.

**LSTM Architecture**  
The LSTM input layer took chord notes (thus 12 nodes), was fully connected to optional hidden layers, and then was fully connected to an output layer with 24 LSTM nodes corresponding to melody notes. The highest melody probability is chosen; if it's above some value, that note is played.

To generate new music, any chord sequence can be input and the resulting output can be converted to MIDI. Many architectures were tested; the best had 2 hidden layers with 9 and 18 LSTM units, respectively.

**Results**  
The generated music was evaluated with human listeners:
* LSTM melodies were preferred over those by humans in 27% of cases (including testing on chords the LSTM had never seen).
* LSTM melodies were differentiable from human compositions in 60% of cases.

Future work may want to pursue a larger training dataset size, more acute compositional goals and tailored data, and another data representation style.  
*-Fin 1/25/2019-*


## 5. Hilscher et al. (2018)

**Summary**  
This work implemented a char-RNN to generate polyphonic classical piano pieces. They used a MIDI dataset of classical music, specifically:  
* 194 Bach pieces, 1.4M characters
* 39 Mozart pieces, 0.4M characters
* 335 tracks by multiple composers, pianomidide, filtered down to 132 tracks with 0.8M characters

**Data Representation**  
They quantized the MIDI data to 16th notes, then assigned each pitch its own ASCII character. The data was augmented to normalize tempo as well as transposed up 0-5 semitones. The characters "on" for each time step were concatenated together and then separated from the next time step by a space. These characters were then 1/0 on/off encoded into vectors.

**Architecture**  
Their char-RNN used a many-to-one architecture, with many input time steps were giving a single output time step. The final layer was a dense softmax which gave probabilities for each character. A temperature value was used to tune how diverse the output notes were.

The best architecture used a single LSTM layer of 512 units, sequence length of 100, Mozart data fully normalized/transposed, batch shuffling, with categorical cross entropy loss, Adam optimizer, 0.001 learning rate, and validation split of 0.2.

**Results/Future Work**  
Wave signal analysis was done to show that the network was not simply reproducing the training data. Temperature experiments showed that the network had not seen enough data. Overall their model achieved 90% training and 70% validation accuracy. Their results can be seen [here](https://yellow-ray.de/~moritz/midi_rnn/examples.html).

Future work may want to consider Seq2Seq, further augmentations to different keys, training on all data at once, other music notations, incorporating MIDI velocity (note dynamics), multiple instruments, or textual pattern matching to measure overfitting.  
*-Fin 1/26/2019-*

## 6. Agarwala et al. (2017)

**Summary**  
This work tested multiple models' abilities to generate syntactically correct monophonic music. The algorithms tested were Continuous Bag-of-Words (CBOW), Char-RNN, Sequence-to-Sequence (Seq2Seq), and Generative Adversarial Networks (GANs). Of these, only Char-RNN and Seq2Seq produced noteworthy results. Code is [here](https://github.com/yinoue93/CS224N_proj).

**Data Representation**  
This work used ABC data, a discrete character-based notation which forces models to construct syntactically correct music. They collected 34,000 ABC examples from abcnotation.com and thesession.com. These were transposed to 4 random keys, encoded into integer representations, which were fed into the embedding layers of the successful models. The data was split 80/10/10 into train/test/dev sets.

**Architecture Specifics**  
Char-RNN: A hidden layer of 200 LSTM cells with 0.2 dropout and embedding size of 20 were chosen. Softmax was used to predict the next character. An input window of size 50 worked best.  
Seq2Seq: Input size was set to 25. In the end of experimentation, a hidden state size 800, embedding size 100, unidirectional cells, and Bahdanau attention were chosen.

**Results**  
The char-RNN was able to compose somewhat passable music, but it had issues with barline placement. The Seq2Seq model was not only able to correctly utilize barlines, but also replicate motifs from a warm start and even compose duets. The char-RNN achieved 59.5% validation accuracy, while the Seq2Seq managed 65%. The latter also fooled 70% of humans into thinking its music was human-composed.  
*-Fin 1/27/2019-*

## 7. Brunner et al. (2017)

**Summary**  
This work implemented an LSTM-based model which generates polyphonic compositions with long-term structure. There are two steps:
1. A chord LSTM generates a chord progression based on learned chord embeddings.
2. A polyphonic LSTM generates music from this chord progression.

Their approach generalized the circle of fifths from training data in its chord embedding.

**Data Representation**  
They used the Lakh MIDI Dataset, which has 114,988 MIDI songs. They analyzed songs' key by their 7 most common notes to find that 86,711 were in major or a relative minor. These were all shifted to C; only notes in the range $\text{C2}-\text{C6}$ were considered.

They then extracted chords by computing the three most-used notes in each bar of each song. They gave the top 50 most-common chords unique integer IDs and the rest an "unknown" tag. These were then encoded as vectors and given unique real-number embeddings of size 10. Quantization was set to 8 time steps per measure.

**Model Architecture**  
Chord LSTM: This model learned those chord embeddings. From this layer, they used a hidden layer with 256 LSTM cells followed by a $\text{softmax}$ activation. The output corresponded to a vector of probabilities for the next chord. Training used cross-entropy as loss, Adam optimizer, $10^{-5}$ initial learning rate, and 80,000 of the shifted songs for 4 epochs. To generate new progressions, they seed the model and then sample output probabilities with temperature. This is fed in and the cycle repeats.

Polyphonic LSTM: This LSTM received vectors of piano rolls (1/0 on/off made with the pretty_midi library) with these appended features:
1. Embedded chord vector of the next time step.
2. Embedded chord vector of the chord following that chord.
3. A binary counter from 0 to 7 each bar.

The input is fed into an LSTM with 512 hidden cells and $\text{sigmoid}$ activation. The output at each time step is the probabilities for each note being played. Training used cross entropy between outputs and the ground truth, Adam, initial learning rate of $10^{-6}$, and only 10,000 songs for 4 epochs.

To generate a new song, the polyphonic LSTM is seeded with the piano roll and corresponding chords. The next step is sampled and the number of notes to play at any one time is limited. To deal with ambiguous note endings/re-attacks, they consider all consecutive same notes as held notes. At barlines, all played notes are re-attacked.

**Results and Future Work**  
The chord LSTM learned embeddings that, when shifted with PCA, show the circle of fifths. The results contain chord progressions typical in western music. Temperature can be used to control this and the melodic variety.

Future work may want to address:  
* Data representations for held notes only exist for monophonic models.
* Another level to their system's hierarchy might guide the construction of verse/bridge/chorus structures.
* The Lakh MIDI Dataset includes artist and genre metadata. Perhaps a future system could use this genre feature.

*-Fin 1/27/2019-*